![ngo_project_image](ngo_project_image.jpg)

GoodThought NGO has been a catalyst for positive change, focusing its efforts on education, healthcare, and sustainable development to make a significant difference in communities worldwide. With this mission, GoodThought has orchestrated an array of assignments aimed at uplifting underprivileged populations and fostering long-term growth.

This project offers a hands-on opportunity to explore how data-driven insights can direct and enhance these humanitarian efforts. In this project, you'll engage with the GoodThought PostgreSQL database, which encapsulates detailed records of assignments, funding, impacts, and donor activities from 2010 to 2023. This comprehensive dataset includes:

- **`Assignments`:** Details about each project, including its name, duration (start and end dates), budget, geographical region, and the impact score.
- **`Donations`:** Records of financial contributions, linked to specific donors and assignments, highlighting how financial support is allocated and utilized.
- **`Donors`:** Information on individuals and organizations that fund GoodThought’s projects, including donor types.

Refer to the below ERD diagram for a visual representation of the relationships between these data tables:
<img src="erd.png" alt="ERD" width="50%" height="50%">


You will execute SQL queries to answer two questions, as listed in the instructions. Good luck!


In [18]:
SELECT	
	a.assignment_name,
	a.region, 
	ROUND(SUM(d.amount), 2) AS rounded_total_donation_amount,
	d2.donor_type
FROM assignments AS a
INNER JOIN donations as d
	on a.assignment_id = d.assignment_id
INNER JOIN donors as d2
	on d.donor_id = d2.donor_id
GROUP BY a.assignment_name, a.region, d2.donor_type
ORDER BY rounded_total_donation_amount DESC
LIMIT 5;


,assignment_name,region,rounded_total_donation_amount,donor_type
0,Assignment_3033,East,3840.66,Individual
1,Assignment_300,West,3133.98,Organization
2,Assignment_4114,North,2778.57,Organization
3,Assignment_1765,West,2626.98,Organization
4,Assignment_268,East,2488.69,Individual


In [19]:
-- top_regional_impact_assignments
SELECT 
	a.assignment_name,
	a.region,
	ROUND(SUM(d1.amount),2) AS rounded_total_donation_amount,
	d2.donor_type
FROM assignments AS a
INNER JOIN donations AS d1 
	ON a.assignment_id = d1.assignment_id
INNER JOIN donors AS d2
	ON d1.donor_id = d2.donor_id
GROUP BY a.assignment_name, a.region, d2.donor_type
ORDER BY rounded_total_donation_amount DESC
LIMIT 5;

,assignment_name,region,rounded_total_donation_amount,donor_type
0,Assignment_3033,East,3840.66,Individual
1,Assignment_300,West,3133.98,Organization
2,Assignment_4114,North,2778.57,Organization
3,Assignment_1765,West,2626.98,Organization
4,Assignment_268,East,2488.69,Individual


In [20]:
-- top_regional_assignments

WITH donation_counts AS (
    SELECT
        assignment_id,
        COUNT(donation_id) AS num_total_donations
    FROM
        donations
    GROUP BY
        assignment_id
),
ranked_assignments AS (
    SELECT
        a.assignment_name,
        a.region,
        a.impact_score,
        dc.num_total_donations,
        ROW_NUMBER() OVER (PARTITION BY a.region ORDER BY a.impact_score DESC) AS rank_in_region
    FROM
        assignments a
    JOIN
        donation_counts dc ON a.assignment_id = dc.assignment_id
    WHERE
        dc.num_total_donations > 0
)
SELECT
    assignment_name,
    region,
    impact_score,
    num_total_donations
FROM
    ranked_assignments
WHERE
    rank_in_region = 1
ORDER BY
    region ASC;

,assignment_name,region,impact_score,num_total_donations
0,Assignment_316,East,10.00,2
1,Assignment_2253,North,9.99,1
2,Assignment_3547,South,10.00,1
3,Assignment_2794,West,9.99,2
